In [3]:
import argparse
import time
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, transforms
from resnet_cifar10_ttq import resnet18
from squeezenet import SqueezeNet
from alexnet import AlexNet
from vgg import VGG

%load_ext autoreload
%autoreload

In [4]:
def get_data(batch_size, test_batch_size):
    normalize = transforms.Normalize((0.491399689874, 0.482158419622, 0.446530924224), (0.247032237587, 0.243485133253, 0.261587846975))
    train_transform = transforms.Compose((transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=2), transforms.ToTensor(), normalize))
    test_transform = transforms.Compose((transforms.ToTensor(), normalize))

    loader_args = {'num_workers': 4}
    if use_cuda:
        loader_args.update({'pin_memory': True})

    data_location = 'data/cifar10'

    trainset = datasets.CIFAR10(root=data_location, train=True, download=True, transform=train_transform)
    trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size=batch_size, **loader_args)

    testset = datasets.CIFAR10(root=data_location, train=False, download=True, transform=test_transform)
    testloader = torch.utils.data.DataLoader(testset, shuffle=False, batch_size=test_batch_size, **loader_args)

    classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    return trainloader, testloader, classes

In [17]:
train_batch_size = 256
test_batch_size = 1024
epochs = 10
lr = 1e-3
momentum = 0.9
weight_decay = 1e-6
log_interval = 20
ttq = True

use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('warning: cuda not available')
    
trainloader, testloader, classes = get_data(train_batch_size, test_batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
model = SqueezeNet(version=1.1, num_classes=10, small_input=True, use_ttq=ttq)
# model = resnet18(use_ttq=True)
# model = AlexNet(use_ttq=True, num_classes=10)

if use_cuda:
    model.cuda()

criterion = F.cross_entropy
optimizer = optim.Adam(model.parameters(), lr=lr)

model.load_state_dict(torch.load('models/squeezenet_cifar.pth'))

In [16]:
def train(epoch):
    model.train()
    running_loss = 0
    running_total = 0
    # correct = 0
    for i, (inputs, labels) in enumerate(trainloader):
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # forward + backward + optimize
        outputs = model(inputs)
        # pred = outputs.data.max(1, keepdim=True)[1]
        # correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        running_total += len(inputs)
        num = i + 1
        if num % log_interval == 0 or num == len(trainloader):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, running_total, len(trainloader.dataset),
                100 * running_total / len(trainloader.dataset), running_loss / num))

def test():
    model.eval()
    correct = 0
    running_loss = 0
    for inputs, labels in testloader:
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        outputs = model(inputs)
        running_loss += criterion(outputs, labels, size_average=False).data[0]
        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max
        correct += pred.eq(labels.data.view_as(pred)).cpu().sum()

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        running_loss / len(testloader.dataset), correct, len(testloader.dataset),
        100 * correct / len(testloader.dataset)))

for epoch in range(1, epochs + 1):
    start = time.time()
    train(epoch)
    delta = time.time() - start
    print('{:.2f}s/epoch'.format(delta))
    torch.save(model.state_dict(), 'model.pth')
    test()

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    outputs = model(Variable(images.cuda(), volatile=True))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels.cuda()).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Train Epoch: 1 [5120/50000 (10%)]	Loss: 0.736911
Train Epoch: 1 [10240/50000 (20%)]	Loss: 0.738652
Train Epoch: 1 [15360/50000 (31%)]	Loss: 0.738221
Train Epoch: 1 [20480/50000 (41%)]	Loss: 0.743175
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.738504
Train Epoch: 1 [30720/50000 (61%)]	Loss: 0.746025
Train Epoch: 1 [35840/50000 (72%)]	Loss: 0.744704
Train Epoch: 1 [40960/50000 (82%)]	Loss: 0.745836
Train Epoch: 1 [46080/50000 (92%)]	Loss: 0.744851
Train Epoch: 1 [50000/50000 (100%)]	Loss: 0.748183
26.78s/epoch

Test set: Average loss: 0.8427, Accuracy: 7202/10000 (72%)

Train Epoch: 2 [5120/50000 (10%)]	Loss: 0.781159
Train Epoch: 2 [10240/50000 (20%)]	Loss: 0.755777
Train Epoch: 2 [15360/50000 (31%)]	Loss: 0.746150
Train Epoch: 2 [20480/50000 (41%)]	Loss: 0.752310
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.752220
Train Epoch: 2 [30720/50000 (61%)]	Loss: 0.751435
Train Epoch: 2 [35840/50000 (72%)]	Loss: 0.750232
Train Epoch: 2 [40960/50000 (82%)]	Loss: 0.752298
Train Epoch: 2 [46080/50000

In [ ]:
#  Resnet18
## Train loss: 0.508
## Test Loss: 0.69   Test Accuracy: 0.77

# Resnet18 TTQ
## Train loss: 0.30
## Test Loss: 0.49   Test Accuracy: 0.83

# SqueezeNet
## Train loss: 0.704571
## Test loss: 0.8143    Test Accuracy: 0.73